In [1]:
import os
import shutil

def recreate_output_folder(output_path):
    """
    output 폴더를 삭제하고 새로 생성합니다.

    Args:
        output_path (str): output 폴더 경로
    """
    try:
        # output 폴더가 존재하면 삭제
        if os.path.exists(output_path):
            shutil.rmtree(output_path)  # 하위 파일 및 폴더 모두 삭제
            print(f"기존 output 폴더({output_path})를 삭제했습니다.")

        # output 폴더 생성
        os.makedirs(output_path)
        print(f"새로운 output 폴더({output_path})를 생성했습니다.")

    except Exception as e:
        print(f"오류 발생: {e}")

# 사용 예시
output_path = "output"  # output 폴더 경로
recreate_output_folder(output_path)

기존 output 폴더(output)를 삭제했습니다.
새로운 output 폴더(output)를 생성했습니다.


In [2]:
import json
import torch
import librosa
import torch.nn.functional as F
from transformers import ClapModel, ClapProcessor

import os
import random

# 설정
input_json_path = "/data/dataset/stage1_valid_05.json"  # 입력 JSON 파일 경로
output_json_path = "output.json"  # 출력 JSON 파일 경로
similarity_threshold = 0.1  # 유사도 임계값 (조정 필요)

# Hugging Face CLAP 모델 및 프로세서 로드
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
model = ClapModel.from_pretrained("laion/clap-htsat-unfused").to(device)
model.eval()

dirname = os.path.dirname(input_json_path)

# JSON 데이터 로드
with open(input_json_path, "r") as f:
    data = json.load(f)

max_annotations = 1000

filtered_annotations = [
    entry for entry in data["annotation"] if entry["task"] == "audiocaption"
]
# annotations = filtered_annotations

# if max_annotations is not None:
#     annotations = random.sample(filtered_annotations, min(len(filtered_annotations), max_annotations))

# data에서 filtered_annotations 제거
data["annotation"] = [
    entry for entry in data["annotation"] if entry["task"] != "audiocaption"
]

for entry in filtered_annotations:
    audio_path = os.path.join(dirname, entry["path"])
    text = entry["text"]
    
    try:
        # 오디오 파일 로드 (48kHz로 리샘플링)
        waveform, sr = librosa.load(audio_path, sr=48000, mono=True)
        
        # 오디오 전처리
        inputs = processor(audios=[waveform], return_tensors="pt", sampling_rate=sr)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        
        # 모델 추론
        with torch.no_grad():
            audio_embed = model.get_audio_features(**inputs).squeeze()  # (dim,)
            text_inputs = processor(text=[text], return_tensors="pt")
            text_inputs = {key: val.to(device) for key, val in text_inputs.items()}
            text_embed = model.get_text_features(**text_inputs).squeeze()  # (dim,)

            # 코사인 유사도 계산
            similarity = F.cosine_similarity(audio_embed.unsqueeze(0), text_embed.unsqueeze(0)).item()

        # 임계값 체크
        if similarity >= similarity_threshold:
            filtered_annotations.append(entry)
            print(f"유지: {audio_path} - 유사도: {similarity:.4f}")
        else:
            # 심볼릭 링크 생성 시 유사도 값 추가
            basename = os.path.basename(audio_path)
            similarity_str = f"{similarity:.4f}"  # 유사도 값을 문자열로 변환 (소수점 4자리까지)
            text_str = f"{text}"
            target_link = os.path.join("output", f"{similarity_str}_{text_str}_{basename}") # 파일 이름에 유사도 추가
            print(f"제거: {audio_path} - 유사도: {similarity:.4f}")

            if not os.path.exists(target_link):
                os.symlink(os.path.abspath(audio_path), target_link)
    except Exception as e:
        print(f"오디오 처리 실패: {audio_path} - {str(e)}")
        continue

# 결과 저장
data["annotation"].extend(filtered_annotations)
with open(output_json_path, "w") as f:
    json.dump(data, f, indent=4)

/root/miniconda3/envs/salmonn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


유지: /data/dataset/WavCaps/FreeSound/180000-210000/571320.flac - 유사도: 0.5461
유지: /data/dataset/WavCaps/FreeSound/0-30000/54467.flac - 유사도: 0.1899
유지: /data/dataset/WavCaps/AudioSet_SL/YbrEMnFXsTqo.flac - 유사도: 0.3787
유지: /data/dataset/WavCaps/FreeSound/30000-60000/76327.flac - 유사도: 0.4369
유지: /data/dataset/WavCaps/AudioSet_SL/YksRZ3HG7Tmo.flac - 유사도: 0.2996
유지: /data/dataset/WavCaps/AudioSet_SL/Y0DizopdPMBw.flac - 유사도: 0.4184
유지: /data/dataset/WavCaps/AudioSet_SL/Yu5MyA3RPp7w.flac - 유사도: 0.4292
유지: /data/dataset/WavCaps/FreeSound/150000-180000/442206.flac - 유사도: 0.5851
유지: /data/dataset/WavCaps/FreeSound/90000-120000/261146.flac - 유사도: 0.3678
유지: /data/dataset/WavCaps/BBC_Sound_Effects/NHU05065049.flac - 유사도: 0.4021
유지: /data/dataset/WavCaps/FreeSound/180000-210000/570841.flac - 유사도: 0.3228
유지: /data/dataset/WavCaps/AudioSet_SL/Ybunyg4DRxJk.flac - 유사도: 0.4748
유지: /data/dataset/WavCaps/AudioSet_SL/Y3LcX3xWAgSk.flac - 유사도: 0.4584
유지: /data/dataset/WavCaps/FreeSound/180000-210000/527434.fla